# Shekel's Function

The objective of this project is to minimize Shekel's function using parameters from De Jong function #5:

$$f(\vec{x}) = \left(0.002 + \sum_{i=1}^{25}\frac{1}{i + (x_1 - a_{1i})^6 + (x_2 - a_{2i})^6}\right)^{-1}$$

where $$
\textbf{a} =
 \begin{pmatrix}
    -32 & -16 & 0 & 16 & 32 & -32 & \ldots & 0 & 16 & 32 \\
    -32 & -32 & -32 & -32 & -32 & -16 & \ldots & 32 & 32 & 32
 \end{pmatrix}$$

The number of dimensions is 2 and the input domain is $-65.536 \le x_i \le 65.536$ for $i=1,2$. 










In [77]:
#import modules
import numpy as np
from enum import Enum

## Genetic Algorithm Goals

Use the Optimization_Goal class as an enum type in order to define a goal variable as either min or max.

```
# define a goal as min
goal = Optimization_Goal.MINIMIZE
# define a goal as max
goal = Optimization_Goal.MAXIMIZE
```

In [85]:
 class Optimization_Goal(Enum):
  MINIMIZE = 1
  MAXIMIZE = 2

## Genetic Algorithm Problems

Genetic algorithms are suited for optimization problems. Use the Problem class to define a problem subclass that can be pass can used as a genetic algorithm parameter. 

In [105]:
class Problem():
  """The Problem class defines the problems contains the definitions of
  of problems. Problems include any information such as the optimization
  goal (min/max), objective function, input dimension, domain/range constraints, 
  and more."""
  class De_Jong_Function_5():
    """De Jong function #5 is variation of Shekel's foxeholes problem involving
    25 local minima, 2 dimensions, and predefined constants.
    This problem is suited for multimodal optimization."""
    dimensions=2
    goal=Optimization_Goal.MINIMIZE
    upper_bounds = [65.536, 65.536]
    lower_bounds = [-65.536, 65.536]
    constants_array = np.array(
        [[-32., -16., 0., 16., 32., -32., -16., 0., 16., 32., -32., -16., 0., 
          16., 32., -32., -16., 0., 16., 32., -32., -16., 0., 16., 32.],
         [-32., -32., -32., -32., -32., -16., -16., -16., -16., -16., 0., 0., 
          0., 0., 0., 16., 16., 16., 16., 16., 32., 32., 32., 32., 32.]]
      )
    def objective_function(self, X):
      # array representing the 1st to 25th elements of the summation
      array_i = np.arange(1, self.constants_array.shape[1]+1, step=1)
      results = (0.002 + np.sum(1/(array_i 
                          + (X[:,[0]] - self.constants_array[0])**6
                          + (X[:, [1]] - self.constants_array[1])**6), axis=1))**-1
      return results    

In [106]:
problem = Problem.De_Jong_Function_5()

In [109]:
x = np.array([[1,2], [1,2]])
problem.objective_function(x)

array([67.47209734, 67.47209734])

[[4 5 6 7]
 [5 6 7 8]]


1. Write a brief (no more than a paragraph) note on the importance of the choice of parameter settings in evolutionary algorithms. Or argue, with well-articulated, precise reasons (one paragraph only), that parameters do not really matter in EC.

I would say that when working with evolutionary algorithms, diversity is very important for the growth and success of the evolutionary algorithms. There should not be one indiviudual within the population that over takes the other. If that would happen then there would be stagnation within the growth of the algorithms. In order to prevent such an event of happening and encourage growth and diversity, it is important to focus on the choice of parameters settings in evolutionary algorithms. If good parameters were choosen, there will be higher chances or growth and diversity to happen and the inverse would result in a higher chance of stangnation.